In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW2") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
from preprocessing.Preprocessor import Preprocessor
from metrics.metrics import Metrics
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
pp = Preprocessor(spark)
train = pp.to_word2vec('Cleaned Data/train.csv', vector_size=128, remove_stop_words = False)
test =  pp.to_word2vec('Cleaned Data/test.csv')

In [34]:
train = train.sample(False, 0.6)

In [35]:
train.cache()
test.cache()

DataFrame[features: vector, label: int]

In [36]:
rf = DecisionTreeClassifier(featuresCol='features',labelCol='label',maxDepth=30,maxBins=10)

In [37]:
%%time
rf_model = rf.fit(train)

CPU times: user 14.5 ms, sys: 4.77 ms, total: 19.3 ms
Wall time: 606 ms


In [38]:
predictions = rf_model.transform(test)

In [39]:
test_m = Metrics()
test_pred_and_labels = predictions.select('prediction', 'label')
test_m.f1(test_pred_and_labels)
test_m.confusion_matrix(test_pred_and_labels)

F1 score for given DataFrame: 48.0

                Predictions
            negative    neutral     positive    
negative    1395        704         2032        
neutral     411         281         787         
positive    2557        2563        5663        
(True labels are on the side)

